In [ ]:
from IPython.display import display
import ipywidgets as ipw
import widgets
import utils
import os

In [ ]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
DATA_MODEL = utils.read_yaml("/home/jovyan/aiida-openbis/Notebooks/Metadata_Schemas_LinkML/materialMLinfo.yaml")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])

project_selector = widgets.ProjectSelectionWidget()
project_selector.load_dropdown_box()

measurements_selector = widgets.ObjectMultipleSelectionWidget("Measurements")
simulations_selector = widgets.ObjectMultipleSelectionWidget("Simulations")
software_selector = widgets.ObjectMultipleSelectionWidget("Software")
software_selector.load_selector(["SOFTWARE"])
code_selector = widgets.ObjectMultipleSelectionWidget("Code")
code_selector.load_selector(["CODE"])

analysis_widgets = widgets.OpenbisObjectWidget("Analysis", DATA_MODEL)

analysis_support_files = ipw.FileUpload(multiple = True)

increase_buttons_size = utils.HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))
create_button = utils.Button(
    description = '', disabled = False, button_style = '', tooltip = 'Save', 
    icon = 'save', layout = ipw.Layout(width = '100px', height = '50px')
)
quit_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)
save_close_buttons_hbox = ipw.HBox([create_button, quit_button])

In [ ]:
def close_notebook(b):
    display(utils.Javascript(data = 'window.location.replace("home.ipynb")'))

def load_project_objects_metadata(change):
    if project_selector.dropdown.value != -1:
        simulation_types = ["GEOMETRY_OPTIMISATION", "BAND_STRUCTURE", "VIBRATIONAL_SPECTROSCOPY", "PDOS", "MEASUREMENT_SESSION"]
        simulations_selector.load_selector(simulation_types, project_selector.dropdown.value)
        measurements_selector.load_selector(["MEASUREMENT_SESSION"], project_selector.dropdown.value)

def create_analysis_openbis(b):
    if project_selector.dropdown.value == -1:
        display(utils.Javascript(data = "alert('Select a project.')"))
        return
    
    project_permid = project_selector.dropdown.value
    project_analysis_collection = OPENBIS_SESSION.get_collections(project = project_permid, code = "ANALYSIS_COLLECTION")
    analysis_parents = []
    analysis_parents.extend(list(measurements_selector.selector.value))
    analysis_parents.extend(list(simulations_selector.selector.value))
    
    if len(project_analysis_collection) == 0:
        analysis_collection = utils.create_openbis_collection(
            OPENBIS_SESSION, 
            project = project_selector.dropdown.value, 
            code = "ANALYSIS_COLLECTION", type = "COLLECTION", props = {"name": "Analysis"}
        )
    else:
        analysis_collection = project_analysis_collection[0]

    object_properties = utils.get_object_widget_values(analysis_widgets.properties_widgets)
    object_properties["softwares"] = list(software_selector.selector.value)
    object_properties["codes"] = list(code_selector.selector.value)
    
    results_object = utils.create_openbis_object(
        OPENBIS_SESSION, type="ANALYSIS", collection=analysis_collection, 
        props = object_properties, parents = analysis_parents
    )
    utils.upload_datasets(OPENBIS_SESSION, results_object, analysis_support_files, "RAW_DATA")
    
    display(utils.Javascript(data = "alert('Upload successful!')"))
    
    # Reset widgets
    simulations_selector.selector.options = []
    measurements_selector.selector.options = []
    software_selector.selector.value = []
    project_selector.dropdown.value = -1

# Create analysis

## Select project

In [ ]:
display(project_selector.dropdown_boxes)
project_selector.dropdown.observe(load_project_objects_metadata, names = 'value')

## Select simulations

In [ ]:
display(simulations_selector)

## Select measurements

In [ ]:
display(measurements_selector)

## Select software

In [ ]:
display(software_selector)

## Select code

In [ ]:
display(code_selector)

## Properties

In [ ]:
display(analysis_widgets)

## Support files

In [ ]:
display(analysis_support_files)

## Save results

In [ ]:
display(save_close_buttons_hbox)
display(increase_buttons_size)
create_button.on_click(create_analysis_openbis)
quit_button.on_click(close_notebook)